# Model Tuning with Grid Search

## Objectives

- Explain what hyperparameters are
- Describe the purpose of grid searching
- Implement grid searching for the purposes of model optimization

![](https://imgs.xkcd.com/comics/machine_learning.png)

## Hyperparameters

Many of the models we have looked at are really *families* of models in the sense that they make use of **hyperparameters**.

Thus for example the $k$-nearest-neighbors algorithm allows us to make:

- a 1-nearest-neighbor model
- a 2-nearest-neighbors model
- a 3-nearest-neighbors model
- etc.

Or, for another example, the decision tree algorithm allows us to make:

- a classifier that branches according to information gain
- a classifier that branches according to Gini impurity
- a regressor that branches according to mean squared error
- etc.

Depending on the sort of problem and data at hand, it is natural to experiment with different values of these hyperparameters to try to improve model performance.

> We can think of these **hyperparameters** as _dials_ of the base model

<img src='https://cdn.dribbble.com/users/947489/screenshots/4522230/0001-0128.gif' width=500>

### Difference from Parametric / Non-Parametric Models

Contrast the notion of hyperparameters with the distinction between **parametric** and **non-parametric** models.

A linear regression model is parametric in the sense that we start with a given model *form* and we then search for the optimal parameters to fill in that form. But *those* parameters are not the sort we might tweak for the purposes of improving model performance! On the contrary, there is **one** best set of parameters, and the training of the model is a matter of finding those optimal values.

## Scenario: Cat in the Dat(a)

Let's revisit our cat in the data kaggle competition: https://www.kaggle.com/c/cat-in-the-dat-ii

In [ ]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [ ]:
# Grab, then explore data
df = pd.read_csv("data/cat_in_the_dat2_train.csv", index_col='id')

In [ ]:
# Define our X and y

X = df.drop(columns=['target'])
y = df['target']

# and train test split - to create our val holdout set!
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1,
                                                  random_state=0)

### Preprocessor Pipeline

Let's bring back our preprocessor pipeline we built:

In [ ]:
# First defining our overall used columns - those with less than 30 uniques
used_cols = [col for col in X_train.columns if len(X_train[col].unique()) < 30]

# Defining our object columns
obj_cols = [col for col in used_cols if X_train[col].dtype == 'O']
# And our object transformer
obj_transformer = Pipeline(steps=[
    ("obj_imputer", SimpleImputer(strategy = 'constant', fill_value='WHATEVER')),
    ('ohe', OneHotEncoder(drop=['WHATEVER']*len(obj_cols)))
])

# Now defining our numeric columns
num_cols = [col for col in used_cols if X_train[col].dtype != 'O']
# And our numeric transformer
num_transformer = Pipeline(steps = [
    ('num_imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Putting them together into a preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num_trans", num_transformer, num_cols),
    ('obj_trans', obj_transformer, obj_cols)
])

### Trying Different Models & Values

Let's explore three different model types, using our preprocessor pipeline and evaluating using `cross_validate`:

#### Logistic Regression

#### Decision Tree

#### Your Choice: KNN or Naive Bayes?

## Automatically Searching for Optimal Hyperparameters with Grid Search

It's not a bad idea to experiment with the values of your models' hyperparameters a bit as you're getting a feel for your models' performance. But there are more systematic ways of going about the search for optimal hyperparameters. One method of hyperparameter tuning is **grid searching**. 

The idea is to build multiple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance. 

For example, for KNN:

    n_neighbors | metric      | weights
    ______________________________________
    1           | 'minkowski' | 'uniform'
    3           | 'manhattan' | 'distance'
    5           |


Scikit-Learn has a [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

### `GridSearchCV`

Note! When using `GridSearchCV` in SKLearn with a pipeline, you need to use the nicknames we give to tell it where these hyperparameters are being tested!

Resource: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

In [ ]:
# Import GridSearchCV


In [ ]:
# Choose one model type to optimize, and set up the base pipeline

clf = None

In [ ]:
# Define the parameter grid for one model type

param_grid = {
    None
}

**Question: How many models will we be constructing with this grid?**

In [ ]:
# Initialize the grid search object with five-fold cross-validation

gs = None

# Then fit it to our X_train and y_train

In [ ]:
# Check out the best parameters
gs.best_params_

In [ ]:
# Check out the best score
gs.best_score_

In [ ]:
# We can grab the best estimator out, as a fit model!
gs.best_estimator_

In [ ]:
# A bigger picture of our results
gs.cv_results_

In [ ]:
# Which we can throw into a dataframe if we like
pd.DataFrame(gs.cv_results_)

### Choice of Grid Values

Which values should you pick for your grid? Intuitively, you should try both "large" and "small" values, but of course what counts as large and small will really depend on the type of hyperparameter.

- For a k-nearest neighbors model, 1 or 3 would be a small value for the number of neighbors and 15 or 17 would be a large value.
- For a decision tree model, what counts as a small `max_depth` will really depend on the size of your training data. A `max_depth` of 5 would likely have little effect on a very small dataset but, at the same time, it would probably significantly decrease the variance of a model where the dataset is large.
- For a logistic regression's regularization constant, you may want to try a set of values that are exponentially separated, like \[1, 10, 100, 1000\].
- **If a grid search finds optimal values at the ends of your hyperparameter ranges, you might try another grid search with more extreme values.**

### Exercise

Do a grid search on a different model type! What are the optimal values for the hyperparameters you've chosen?

# Level Up: Random Searching

It is also possible to search for good hyperparameter values randomly. This is a nice choice if computation time is an issue or if you are tuning over continuous hyperparameters.

### `RandomizedSearchCV` with `LogisticRegression`

In [ ]:
# Import
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
log_reg_grid = {'C': stats.uniform(loc=0, scale=10),
               'l1_ratio': stats.expon(scale=0.2)}

In [ ]:
rs = RandomizedSearchCV(estimator=LogisticRegression(penalty='elasticnet',
                                                    solver='saga',
                                                    max_iter=1000,
                                                    random_state=42),
                        param_distributions=log_reg_grid,
                        random_state=42)

rs.fit(X_train, y_train)

rs.best_params_

# Level Up: More Grid Search Practice

Use a classifier of your choice to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using a grid search, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/phone-train.csv')
phones_test = pd.read_csv('data/phone-test.csv')

In [ ]:
# Your code here